# Protein Protein Interaction Data
**[Work in progress]**

This notebook downloads and standardizes viral-host protein data from IntAct for ingestion into the Knowledge Graph.

Data source: [IntAct](https://www.ebi.ac.uk/intact/query/pubid:IM-27814)

Authors: Kaushik Ganapathy, Eric Yu, Peter Rose (krganapa@ucsd.edu, ery010@ucsd.edu, pwrose@ucsd.edu)

In [1]:
import os
import re
import pandas as pd
from pathlib import Path

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


### Get list of organisms to include in the Knowledge Graph

In [4]:
organisms = pd.read_csv("../../reference_data/Organism.csv", dtype=str)

In [5]:
organisms = organisms[organisms['id'].str.startswith('taxonomy')]
# remove CURIE
organisms['taxonomy'] = organisms['id'].apply(lambda x: x.split(':')[1])
taxonomy_ids = organisms['taxonomy'].unique()

In [6]:
taxonomy_ids

array(['2697049', '1263720', '694009', '443239', '31631', '11137',
       '277944', '12131', '12134', '766791', '693998', '1487703',
       '285949', '9606', '10090', '59477', '608659', '49442', '9974',
       '143292', '71116', '9608', '9685', '9666', '419130', '452646',
       '10036', '9689', '9694'], dtype=object)

### Retrieve interaction data from IntAct

In [7]:
urls = [f'https://www.ebi.ac.uk/intact/export?format=mitab_25&query=taxid%3A{taxon}&negative=false&spoke=false&ontology=false&sort=intact-miscore&asc=false'
        for taxon in taxonomy_ids]

In [8]:
data = pd.concat((pd.read_csv(url, sep='\t', dtype='str') for url in urls))

In [9]:
print('Number of interactions:', data.shape[0])

Number of interactions: 780441


In [10]:
data.head()

,#ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,Taxid interactor B,Interaction type(s),Source database(s),Interaction identifier(s),Confidence value(s)
0,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0410""(3D electron microscopy)",Walls et al. (2020),pubmed:32155444|imex:IM-27846,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25495631|wwpdb:6vyb|wwpdb:6vxx|imex...,intact-miscore:0.94
1,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0114""(x-ray crystallography)",Xia et al. (2020),imex:IM-27873|pubmed:32231345,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25503580|wwpdb:6LXT|imex:IM-27873-1,intact-miscore:0.94
2,uniprotkb:P0DTD1-PRO_0000449632,uniprotkb:P0DTD1-PRO_0000449632,intact:EBI-25475891,intact:EBI-25475891,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,"psi-mi:""MI:0114""(x-ray crystallography)",Kim. et al. (2020),pubmed:32304108|imex:IM-27884,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25504928|imex:IM-27884-1|wwpdb:6vww...,intact-miscore:0.44
3,uniprotkb:P0DTD1-PRO_0000449625,uniprotkb:P0DTD1-PRO_0000449626,intact:EBI-25475871,intact:EBI-25475874,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,psi-mi:p0dtd1-pro_0000449626(display_long)|uni...,"psi-mi:""MI:0410""(3D electron microscopy)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25506373|wwpdb:7btf|wwpdb:6M71|imex...,intact-miscore:0.94
4,uniprotkb:P0DTD1-PRO_0000449625,uniprotkb:P0DTD1-PRO_0000449629,intact:EBI-25475871,intact:EBI-25475885,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,psi-mi:p0dtd1-pro_0000449629(display_long)|uni...,"psi-mi:""MI:0410""(3D electron microscopy)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25506373|wwpdb:7btf|wwpdb:6M71|imex...,intact-miscore:0.86


### Process Data

In [11]:
# change CURIE from wwpdb to pdb to enable resolution by identifiers.org
data['interactionIds'] = data['Interaction identifier(s)'].str.replace('wwpdb', 'pdb')
# represent one-to-many relationship as semicolon separated string
data['interactionIds'] = data['interactionIds'].str.replace('|', ';')
# extract unique interaction id
data['interactionId'] = 'intact:EBI-' + data['interactionIds'].str.extract('intact:EBI-(\d*)')

#### Remove duplicates

In [12]:
data.drop_duplicates(subset=['interactionId'], inplace=True)

In [13]:
print('Number of interactions without duplicates:', data.shape[0])

Number of interactions without duplicates: 446520


In [14]:
data.head()

,#ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,Taxid interactor B,Interaction type(s),Source database(s),Interaction identifier(s),Confidence value(s),interactionIds,interactionId
0,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0410""(3D electron microscopy)",Walls et al. (2020),pubmed:32155444|imex:IM-27846,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25495631|wwpdb:6vyb|wwpdb:6vxx|imex...,intact-miscore:0.94,intact:EBI-25495631;pdb:6vyb;pdb:6vxx;imex:IM-...,intact:EBI-25495631
1,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0114""(x-ray crystallography)",Xia et al. (2020),imex:IM-27873|pubmed:32231345,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25503580|wwpdb:6LXT|imex:IM-27873-1,intact-miscore:0.94,intact:EBI-25503580;pdb:6LXT;imex:IM-27873-1,intact:EBI-25503580
2,uniprotkb:P0DTD1-PRO_0000449632,uniprotkb:P0DTD1-PRO_0000449632,intact:EBI-25475891,intact:EBI-25475891,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,"psi-mi:""MI:0114""(x-ray crystallography)",Kim. et al. (2020),pubmed:32304108|imex:IM-27884,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25504928|imex:IM-27884-1|wwpdb:6vww...,intact-miscore:0.44,intact:EBI-25504928;imex:IM-27884-1;pdb:6vww;p...,intact:EBI-25504928
3,uniprotkb:P0DTD1-PRO_0000449625,uniprotkb:P0DTD1-PRO_0000449626,intact:EBI-25475871,intact:EBI-25475874,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,psi-mi:p0dtd1-pro_0000449626(display_long)|uni...,"psi-mi:""MI:0410""(3D electron microscopy)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25506373|wwpdb:7btf|wwpdb:6M71|imex...,intact-miscore:0.94,intact:EBI-25506373;pdb:7btf;pdb:6M71;imex:IM-...,intact:EBI-25506373
5,uniprotkb:P0DTD1-PRO_0000449626,uniprotkb:P0DTD1-PRO_0000449625,intact:EBI-25475874,intact:EBI-25475871,psi-mi:p0dtd1-pro_0000449626(display_long)|uni...,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,"psi-mi:""MI:0071""(molecular sieving)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25510487|imex:IM-27888-3,intact-miscore:0.94,intact:EBI-25510487;imex:IM-27888-3,intact:EBI-25510487


#### Apply naming conventions

In [15]:
data.rename(columns={'Alt. ID(s) interactor A': 'interactorA', 'Alt. ID(s) interactor B': 'interactorB'}, inplace=True)

#### Extract UniProt accession number and Uniprot protein ids from interactor columns

In [16]:
# uniprot:P0DTD1-PRO_0000449619 -> P0DTD1-PRO_0000449619
data['id_a'] = data['#ID(s) interactor A'].str.replace('uniprotkb:', '')
data['id_b'] = data['ID(s) interactor B'].str.replace('uniprotkb:', '')

# P0DTD1-PRO_0000449619 -> P0DTD1 (UniProt accession number)
data['accessionA'] = data['id_a'].str.split('-PRO', expand=True)[0]
data['accessionB'] = data['id_b'].str.split('-PRO', expand=True)[0]

# Remove isoform id: Q9UBL6-2 -> Q9UBL6
data['accessionA'] = data['accessionA'].str.split('-', expand=True)[0]
data['accessionB'] = data['accessionB'].str.split('-', expand=True)[0]

# ADD CURIE "uniprot" as prefix (see https://registry.identifiers.org/registry/uniprot)
data['accessionA'] = 'uniprot:' + data['accessionA']
data['accessionB'] = 'uniprot:' + data['accessionB']

# P0DTD1-PRO_0000449619 -> PRO_0000449619 (UniProt protein id)
data['proIdA'] = data['id_a'].str.split('-PRO', expand=True)[1]
data['proIdB'] = data['id_b'].str.split('-PRO', expand=True)[1]

# Add CURIE "uniprot.chain" as prefix (see https://registry.identifiers.org/registry/uniprot.chain)
data['proIdA'] = data['proIdA'].str.replace('_', 'uniprot.chain:PRO_')
data['proIdB'] = data['proIdB'].str.replace('_', 'uniprot.chain:PRO_')

In [17]:
# extract text descriptions in  parenthesis
data['detectionMethod'] = data['Interaction detection method(s)'].str.extract('.*\((.*)\).*') 
data['interactionType'] = data['Interaction type(s)'].str.extract('.*\((.*)\).*') 

In [18]:
# set no-numerical values to ""
data['confidenceValue'] = data['Confidence value(s)'].str.split(':', expand=True)[1]
data['confidenceValue'] = data['confidenceValue'].apply(lambda s: s if s and s.replace('.','',1).isdigit() else '')

In [19]:
data['confidenceValue'].unique()

array(['0.94', '0.44', '0.93', '0.37', '0.70', '0.84', '0.49', '0.36',
       '0.56', '0.99', '0.62', '0.50', '0.53', '0.35', '0.40', '0.27',
       '0.64', '0.59', '0.32', '0.54', '0.80', '0.77', '0.72', '0.71',
       '0.68', '0.46', '0.66', '0.63', '0.87', '0.52', '0.51', '0.73',
       '0.81', '0.67', '0.89', '0.98', '0.91', '0.65', '0.57', '0.74',
       '0.61', '0.75', '0.76', '0.69', '0.43', '0.55', '0.60', '0.58',
       '0.82', '0.78', '0.83', '0.88', '0.96', '0.97', '0.90', '0.95',
       '0.85', '0.86', '0.92', '0.79', '0.41', '0.48', '', '0.28', '0.47',
       '0.31', '0.22', '0.45', '0.23', '0.34'], dtype=object)

#### Extract pubmed id
Example: imex:IM-27912|pubmed:32275855 -> 2275855

In [20]:
data['pubmedId'] = data['Publication Identifier(s)'].str.extract('pubmed:(\d*)')
# add CURIE prefix if pubmedId exists
data['pubmedId'] = data['pubmedId'].apply(lambda s: 'pubmed:' + s if len(s) > 0 else '')

#### Extract taxonomy id
Example: taxid:9606(human)|taxid:9606(Homo sapiens) -> 9606

In [21]:
data['taxonomyIdA'] = data['Taxid interactor A'].str.extract('taxid:(\d*)\(')
data['taxonomyIdB'] = data['Taxid interactor B'].str.extract('taxid:(\d*)\(')

In [22]:
data.fillna('', inplace=True)

In [23]:
data.head()

,#ID(s) interactor A,ID(s) interactor B,interactorA,interactorB,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,Taxid interactor B,Interaction type(s),Source database(s),Interaction identifier(s),Confidence value(s),interactionIds,interactionId,id_a,id_b,accessionA,accessionB,proIdA,proIdB,detectionMethod,interactionType,confidenceValue,pubmedId,taxonomyIdA,taxonomyIdB
0,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0410""(3D electron microscopy)",Walls et al. (2020),pubmed:32155444|imex:IM-27846,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25495631|wwpdb:6vyb|wwpdb:6vxx|imex...,intact-miscore:0.94,intact:EBI-25495631;pdb:6vyb;pdb:6vxx;imex:IM-...,intact:EBI-25495631,P0DTC2,P0DTC2,uniprot:P0DTC2,uniprot:P0DTC2,,,3D electron microscopy,direct interaction,0.94,pubmed:32155444,2697049,2697049
1,uniprotkb:P0DTC2,uniprotkb:P0DTC2,intact:EBI-25474821,intact:EBI-25474821,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,psi-mi:spike_sars2-6(display_long)|uniprotkb:S...,"psi-mi:""MI:0114""(x-ray crystallography)",Xia et al. (2020),imex:IM-27873|pubmed:32231345,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25503580|wwpdb:6LXT|imex:IM-27873-1,intact-miscore:0.94,intact:EBI-25503580;pdb:6LXT;imex:IM-27873-1,intact:EBI-25503580,P0DTC2,P0DTC2,uniprot:P0DTC2,uniprot:P0DTC2,,,x-ray crystallography,direct interaction,0.94,pubmed:32231345,2697049,2697049
2,uniprotkb:P0DTD1-PRO_0000449632,uniprotkb:P0DTD1-PRO_0000449632,intact:EBI-25475891,intact:EBI-25475891,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,psi-mi:p0dtd1-pro_0000449632(display_long)|uni...,"psi-mi:""MI:0114""(x-ray crystallography)",Kim. et al. (2020),pubmed:32304108|imex:IM-27884,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25504928|imex:IM-27884-1|wwpdb:6vww...,intact-miscore:0.44,intact:EBI-25504928;imex:IM-27884-1;pdb:6vww;p...,intact:EBI-25504928,P0DTD1-PRO_0000449632,P0DTD1-PRO_0000449632,uniprot:P0DTD1,uniprot:P0DTD1,uniprot.chain:PRO_0000449632,uniprot.chain:PRO_0000449632,x-ray crystallography,direct interaction,0.44,pubmed:32304108,2697049,2697049
3,uniprotkb:P0DTD1-PRO_0000449625,uniprotkb:P0DTD1-PRO_0000449626,intact:EBI-25475871,intact:EBI-25475874,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,psi-mi:p0dtd1-pro_0000449626(display_long)|uni...,"psi-mi:""MI:0410""(3D electron microscopy)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25506373|wwpdb:7btf|wwpdb:6M71|imex...,intact-miscore:0.94,intact:EBI-25506373;pdb:7btf;pdb:6M71;imex:IM-...,intact:EBI-25506373,P0DTD1-PRO_0000449625,P0DTD1-PRO_0000449626,uniprot:P0DTD1,uniprot:P0DTD1,uniprot.chain:PRO_0000449625,uniprot.chain:PRO_0000449626,3D electron microscopy,physical association,0.94,pubmed:32277040,2697049,2697049
5,uniprotkb:P0DTD1-PRO_0000449626,uniprotkb:P0DTD1-PRO_0000449625,intact:EBI-25475874,intact:EBI-25475871,psi-mi:p0dtd1-pro_0000449626(display_long)|uni...,psi-mi:p0dtd1-pro_0000449625(display_long)|uni...,"psi-mi:""MI:0071""(molecular sieving)",Gao et al. (2020),pubmed:32277040|imex:IM-27888,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,taxid:2697049(SARS-CoV-2)|taxid:2697049(SARS-C...,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0469""(IntAct)",intact:EBI-25510487|

#### Restrict data to the set of currently supported taxonomy ids

In [24]:
data = data[data['taxonomyIdA'].isin(taxonomy_ids) & data['taxonomyIdB'].isin(taxonomy_ids)]

In [25]:
data.shape

(412535, 29)

Remove data with accession numbers that are not UniProt accession numbers

In [26]:
data = data[~(data['id_a'].str.contains(':')) & ~(data['id_b'].str.contains(':'))]

In [27]:
data.shape

(390613, 29)

Remove self-interactions (they make graph display too crowded)

In [28]:
data = data[~(data['accessionA'] == data['accessionB'])]

In [29]:
data.shape

(382232, 29)

In [30]:
data['taxonomyIdA'] = 'taxonomy:' + data['taxonomyIdA']
data['taxonomyIdB'] = 'taxonomy:' + data['taxonomyIdB']

In [31]:
data['source'] = 'IntAct'

In [32]:
data = data[['accessionA', 'accessionB', 'proIdA', 'proIdB', 'taxonomyIdA', 'taxonomyIdB', 'interactionId', 'interactionIds', 'interactionType', 'detectionMethod', 'confidenceValue','pubmedId']]

### Save interaction data

In [33]:
data.to_csv(NEO4J_IMPORT / '01e-ProteinProteinInteraction.csv', index = False)

In [34]:
print('Number of interactions:', data.shape[0])

Number of interactions: 382232


In [35]:
data.sample(5)

,accessionA,accessionB,proIdA,proIdB,taxonomyIdA,taxonomyIdB,interactionId,interactionIds,interactionType,detectionMethod,confidenceValue,pubmedId
284321,uniprot:Q09472,uniprot:Q9BQG0,,,taxonomy:9606,taxonomy:9606,intact:EBI-4374183,intact:EBI-4374183;imex:IM-17006-2,association,anti bait coimmunoprecipitation,0.35,pubmed:21471221
282366,uniprot:Q9NPB6,uniprot:P60953,,,taxonomy:9606,taxonomy:9606,intact:EBI-81907,intact:EBI-81907,physical association,two hybrid,0.75,pubmed:10934474
604731,uniprot:O89100,uniprot:Q13480,,,taxonomy:10090,taxonomy:9606,intact:EBI-8652924,intact:EBI-8652924;mint:MINT-60136,direct interaction,isothermal titration calorimetry,0.44,pubmed:12773374
385654,uniprot:Q5T7P3,uniprot:O43597,,,taxonomy:9606,taxonomy:9606,intact:EBI-10245948,intact:EBI-10245948;imex:IM-23318-6675,physical association,two hybrid prey pooling approach,0.74,pubmed:25416956
393207,uniprot:Q05655,uniprot:P11388,,,taxonomy:9606,taxonomy:9606,intact:EBI-9982335,intact:EBI-9982335;imex:IM-23308-4,physical association,pull down,0.68,pubmed:16611985
